In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn.ensemble
import sklearn.preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns

In [16]:
#Read in data
data = pd.read_csv('../train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
#Remove attributes that are presumed irrelevant and see data
data.drop("PassengerId", axis=1, inplace=True)
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
#Replace strings with their hashes, as sklearn can only handle numeric input
data['Sex'] = data['Sex'].map(lambda x: hash(x))
data['Embarked'] = data['Embarked'].map(lambda x: hash(x))

#Drop NaN values
data.dropna(inplace=True)

In [7]:
#Split into training and testing sets
(train_data, test_data) = train_test_split(data, test_size=0.1, random_state=67854895) 

x_train = train_data.drop('Survived', axis=1)
y_train = train_data['Survived']

x_test = test_data.drop('Survived', axis=1)
y_test = test_data['Survived']

In [10]:
#Returns the classifier's accuracy
def accuracy(classifier) -> float:
    num_correct = 0
    for i in range(len(x_test)):
        pred = classifier.predict([x_test.values[i]])[0]
        correct = y_test.values[i]
        if pred == correct:
                num_correct += 1
    return num_correct / len(x_test)

#Function that returns the false positive and false negative rate of a classifier
def false_rates(classifier) -> (float, float):
    num_negative = 0
    incorrect_negative = 0
    num_positive = 0
    incorrect_positive = 0
    for i in range(len(x_test)):
        pred = classifier.predict([x_test.values[i]])[0]
        correct = y_test.values[i]
        if correct == 0:
            num_negative += 1
            if pred != correct:
                incorrect_negative += 1
        else:
            num_positive += 1
            if pred != correct:
                incorrect_positive += 1
    fp = 0
    if num_negative != 0:
        fp = incorrect_negative / num_negative
    fn = 0
    if num_positive != 0:
        fn = incorrect_positive / num_positive
    return (fp, fn)

In [11]:
classifier = KNeighborsClassifier(n_neighbors=8, weights='distance')
classifier.fit(x_train, y_train)
print(false_rates(classifier))
print(accuracy(classifier))

(0.23809523809523808, 0.26666666666666666)
0.75


In [12]:
test_data = pd.read_csv('../test.csv')
test_data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test_data['Sex'] = test_data['Sex'].map(lambda x: hash(x))
test_data['Embarked'] = test_data['Embarked'].map(lambda x: hash(x))
test_data.fillna(value=-10, axis=1, inplace=True)

output = [{'PassengerId' : 892 + i, 'Survived' : classifier.predict([test_data.values[i]])[0]} for i in range(len(test_data))]
pd.DataFrame(output).to_csv(path_or_buf='KNN2_Output.csv', index=False)